In [1]:
%cd ..

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.utils.vis_utils import model_to_dot
from keras_tqdm import TQDMNotebookCallback
from keras import backend as K
from tqdm import tqdm_notebook

from data import ZeroFilled3DSequence
from unet3d import unet3d

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

In [4]:
n_samples_train = 34742
n_samples_val = 7135

n_volumes_train = 973
n_volumes_val = 199

In [5]:
# generators
train_gen = ZeroFilled3DSequence(train_path, af=4, norm=True)
val_gen = ZeroFilled3DSequence(val_path, af=4, norm=True)

In [6]:
run_params = {
    'n_layers': 4, 
    'pool': 'max', 
    "layers_n_channels": [16, 32, 64, 128], 
    'layers_n_non_lins': 2,
}
n_epochs = 2
run_id = 'vnet_test_2'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'

In [7]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=50)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
lr_on_plat_cback = ReduceLROnPlateau(monitor='val_loss', min_lr=5*1e-5, mode='auto', patience=3)

In [8]:
model = unet3d(input_size=(None, 320, 320, 1), lr=1e-3, **run_params)
# model = unet(input_size=(320, 320, 1))
# print(f'Loading from {chkpt_path}')
# model = load_model(chkpt_path)
print(model.summary(line_length=150))

W0730 16:42:07.172534 139992708126464 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0730 16:42:07.174900 139992708126464 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0730 16:42:07.178053 139992708126464 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0730 16:42:07.456263 139992708126464 deprecation_wrapper.py:119] From /volatile/home/Zacchar

______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_1 (InputLayer)                             (None, None, 320, 320, 1)        0                                                                   
______________________________________________________________________________________________________________________________________________________
conv3d_1 (Conv3D)                                (None, None, 320, 320, 16)       448               input_1[0][0]                                     
______________________________________________________________________________________________________________________________________________________
conv3d_2 (Conv3D)                                (None, None, 320, 320, 16)       6928        

In [ ]:
model.fit_generator(
    train_gen, 
    steps_per_epoch=n_volumes_train, 
    epochs=n_epochs,
    validation_data=val_gen,
    validation_steps=n_volumes_val,
    verbose=0,
    callbacks=[TQDMNotebookCallback(metric_format="{name}: {value:e}"), tboard_cback, chkpt_cback, lr_on_plat_cback],
    max_queue_size=500,
    use_multiprocessing=True,
    workers=35,
)

W0730 16:42:08.773870 139992708126464 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0730 16:42:09.154303 139992708126464 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0730 16:42:11.590510 139992708126464 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0730 16:42:11.592556 139992708126464 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-be

In [ ]:
# # overfitting trials

# data = train_gen[0]
# val_data = val_gen[0]
# model.fit(
#     x=data[0], 
#     y=data[1], 
# #     validation_data=val_data, 
#     batch_size=data[0].shape[0], 
#     epochs=250, 
#     verbose=2, 
#     shuffle=False,
# )